# Train model for detection of geometric patterns

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping


import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [4]:
img_dim = (64,64)
img_channelr = 1

In [5]:
def load_and_preprocess_img(folder_path):
    images = []
    labels = []

    for filename in os.listdir(folder_path): 
        if filename.endswith("jpg"):
            img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_dim)
            #img = img[:, :, np.newaxis]
            img = np.expand(img, axis=-1)

            label = int(filename.split("_")[0])

            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels)

In [6]:
folder_path = "./img_sliced_and_tilted"

In [ ]:
images, labels = load_and_preprocess_img(folder_path)

In [ ]:
images = images / 255
#images = images / 255.0

In [ ]:
x_train, y_train, x_test, y_test = train_test_split(images, labels, test_sixe=0.2, random_state= 42)

In [ ]:
y_train_one_hot = to_categorical(y_train, num_classes = 3)
y_test_one_hot = to_categorical(y_test, num_classes = 3)

In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train_one_hot.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test_one_hot.shape)

### Example of img preprocessed

In [ ]:
plt.figure(figsize=(2, 2))
plt.imshow(x_train[254,:,:])
print(y_train[254])

### Model

In [7]:
model = tf.keras.Sequential([
    layers.Input(shape=(64,64,1)),
    layers.Normalization(),
    layers.Conv2D(16, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.15),
    layers.Conv2D(64, (3, 3),activation='elu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3),activation='elu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.35),
    layers.Flatten(),
    layers.Dense(128, activation='elu'),

    layers.Dense(3, activation='softmax')
])

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate =  0.0001),       
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=tf.keras.metrics.CategoricalAccuracy())

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 64, 64, 1)         3         
 on)                                                             
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                        

In [ ]:
early_stop = EarlyStopping(monitor = "val_loss", patience=10, restore_best_weights = True)

### Run

In [ ]:
history = model.fit(
    x_train,
    y_train_one_hot,
    epochs=20,
    batch_size=16,
    validation_data=(x_test, y_test_one_hot),
    callbacks=[early_stop])

### Results

In [ ]:
plt.plot(history.history['categorical_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')

plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
#plt.legend(loc='upper left')
plt.show()

In [ ]:
class_labels = ["", 
                "", 
                ""
]

In [ ]:
y_pred = model.predict(x_test)

# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert class labels to class labels (no one-hot encoding)
y_true_classes = y_test

# Calculate the confusion matrix
confusion_mat = confusion_matrix(y_true_classes, y_pred_classes)

# Create a nice figure
plt.figure(figsize=(6, 5))
sns.set(font_scale=1.2)  # Adjust font size for readability

# Display the heatmap
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap="Greens", 
            xticklabels= class_labels,
            yticklabels= class_labels,)

plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

### Save model

In [ ]:
#model.save("keras.h5")

# Testning

In [ ]:
def preprocess_image(image, target_height, target_width):
    img = cv2.resize(image, (target_width, target_height))
    img = img.reshape((1, target_height, target_width, 1))  # Reshape to match model input shape
    return img

# def preprocess_image(image, target_height, target_width):
#     img = cv2.resize(image, target_height, target_width)
#     img = np.expand_dims(img, axis=-1)# Reshape to match model input shape
#     return img



def predict_from_image(test_image_preprocessed, model, class_labels):
    prediction = model.predict(test_image_preprocessed)
    predicted_class = np.argmax(prediction)
    predicted_probability = prediction[0, predicted_class]

    # Get the corresponding label for the predicted class
    predicted_label = class_labels[predicted_class]

    print("Prediction probabilities:")
    print(prediction)
    print("\nPredicted class:", predicted_class)
    print("Predicted Label:", predicted_label)
    print("Predicted Probability: {:.2f}%".format(predicted_probability * 100))
    print(predicted_probability)

    return predicted_class, predicted_label, predicted_probability

def test_images(image_paths, model, class_labels, target_height, target_width):
    for image_path in image_paths:
        # Load and preprocess the image
        test_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        test_image_preprocessed = preprocess_image(test_image, target_height, target_width)

        # Display the preprocessed image
        plt.figure(figsize=(2, 2))
        plt.imshow(test_image_preprocessed.squeeze(), cmap='gray')
        plt.show()

        # Predict from the preprocessed image
        print("Image Path:", image_path)
        predicted_class, predicted_label, predicted_probability = predict_from_image(test_image_preprocessed, model, class_labels)
        print("="*30)



image_paths = [
    "C:/Users/Development/Desktop/Patterns/img_for_tests/0_1_slice_4.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/0_2_slice_1.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/1_2_slice_2.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/1_1_slice_3.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/3_1_slice_4.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/3_2_slice_4.jpg",
    "C:/Users/Development/Desktop/Patterns/img_for_tests/3_3_slice_0.jpg",
    "C:/Users/Development/Desktop/geometric_pattern/img_sliced_and_tilted/1_3_slice_83.jpg"
]


def preprocess_image(image):
    img_preprocessed = cv2.resize(img, IMAGE_DIM)
    img_preprocessed = np.expand_dims(img_preprocessed, axis=-1) 

    return img_preprocessed

testing_images = []
for image_path in image_paths:
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_preprocessed = preprocess_image(img)
    testing_images.append(img_preprocessed)

testing_images = np.array(testing_images) / 255.0 

predictions = model.predict(testing_images)

# Display the images, predictions, and prediction probabilities
for i in range(len(image_paths)):
    plt.figure(figsize=(2, 2))
    plt.imshow(testing_images[i].reshape(64, 64), cmap='gray')
    predicted_class = np.argmax(predictions[i])
    plt.title(f"Predicted Class: {predicted_class} ({class_labels[predicted_class]})")
    plt.xlabel("Prediction Probabilities:\n" + "\n".join([f"{class_labels[j]}: {predictions[i][j]:.4f}" for j in range(predictions.shape[1])]))
    plt.show()